Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
* Наименование вакансии.

* Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).

* Ссылку на саму вакансию.

* Сайт, откуда собрана вакансия.

In [1]:
#pip install bs4

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=122021016135370c82427d1792d914854160f9280e4daba50ac1799bc22e05de
  Stored in directory: c:\users\fufik\appdata\local\pip\cache\wheels\73\2b\cb\099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from time import sleep

In [19]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'

url = 'https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_save_area=true&items_on_page=20'

headers = {
    'User-Agent': USER_AGENT
}

response = requests.get(url, headers=headers)

response.status_code

200

In [20]:
def parse_HH(url, headers, result_HH = [], index_page=1):
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Парсинг сайта завершен!")
        return result_HH
    else:
        print('Страница сайта №%d, ссылка на страницу: %s'%(index_page, response.url))
        
    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
        result_HH.append(parse_vacancy_hh(vacancy))
        
    link_next_page = dom.find('a', {'data_qa', 'pager-page'})
    if link_next_page:
        link_next_page = 'https://spb.hh.ru' + link_next_page['href']
    else:
        print('Парсинг сайта завершен!')
        return result_HH
    
    result = parse_HH(link_next_page, headers, result_HH, index_page+1)
    return result_HH

def parse_vacancy_hh(dom_vacancy):
    sleep(0.1)
    vacancy_name = dom_vacancy.find('a').text
    
    vacancy_salary= dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
    if vacancy_salary:
        vacancy_salary = vacancy_salary.text
        min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
    else:
        min_salary, max_salary, currency_salary = None, None, None

    vacancy_link = dom_vacancy.find('a')['href']
  
    return {
        'vacancy_name': vacancy_name,
        'vacancy_salary': vacancy_salary,
        'min_salary': min_salary,
        'max_salary': max_salary,
        'currency_salary': currency_salary,
        'vacancy_link': vacancy_link,
        'vacancy_source': 'hh.ru'
      }
        
    

def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
    list_salary = vacancy_salary_text.replace('\u202f', '').split()
    for i in range(len(list_salary) - 1):
        if list_salary[i] == 'от':
            min_salary = int(list_salary[i + 1])
        elif list_salary[i] == 'до':
            max_salary = int(list_salary[i + 1])
        elif list_salary[i] == '–':
            min_salary = int(list_salary[i - 1])
            max_salary = int(list_salary[i + 1])
    currency_salary = list_salary[-1]

    return min_salary, max_salary, currency_salary


In [21]:
result_HH = parse_HH(url, headers)

Страница сайта №1, ссылка на страницу: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_save_area=true&items_on_page=20
Парсинг сайта завершен!


In [22]:
len(result_HH)

20

In [23]:
result_HH

[{'vacancy_name': 'Разработчик Python',
  'vacancy_salary': 'от 120\u202f000 руб.',
  'min_salary': 120000,
  'max_salary': None,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://spb.hh.ru/vacancy/76076341?from=vacancy_search_list&query=python',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Python разработчик',
  'vacancy_salary': None,
  'min_salary': None,
  'max_salary': None,
  'currency_salary': None,
  'vacancy_link': 'https://spb.hh.ru/vacancy/77409819?from=vacancy_search_list&query=python',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Middle Python developer (Казань)',
  'vacancy_salary': 'от 160\u202f000 руб.',
  'min_salary': 160000,
  'max_salary': None,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://spb.hh.ru/vacancy/76115939?from=vacancy_search_list&query=python',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Аналитик данных',
  'vacancy_salary': 'от 2\u202f000 USD',
  'min_salary': 2000,
  'max_salary': None,
  'currency_salary': 'USD',
  'vacanc